In [ ]:
This version holds sites as major frame work, so a drug dictionary is assumed to be information gleaned only from one site

In [1]:
from bs4 import BeautifulSoup as bf
import requests
import re
import pickle
from collections import defaultdict
import time

In [2]:
def load_soup(url):
    html = requests.get(url).text
    soup = bf(html, 'html5lib')
    return soup

### Go get information about drugs in a category

#### Drugs.com

In [3]:
# each drug has a name and a url_stem
class drug:
    
    def __init__(self):# , name):
        self.name = '' #name
        self.generic = ''
        self.url_drug = '' #{'ddc': '', 'wmd': ''}
        self.url_drug_revs = ''
        self.reviews = []#{'ddc': '', 'wmd': ''}
        self.score = 0     
        self.num_rev = 0 #{'ddc': '', 'wmd': ''}
        self.num_rev_pages = 0


new_drug.__dict__

In [4]:
# ****** ambidexterous 
# for a particular drug
# gets urls for pages of reviews
def revs_url_list(_new_drug, pg_init, pg_n, site):
    if site == 'ddc':
        url_c = ['https://www.drugs.com/comments/', '/?page=', '']
    elif site == 'wmd':
        url_c = ['https://www.webmd.com/drugs/drugreview-', '&pageIndex=', '&sortby=3' ]
#     url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) + url_c[2] for ik in range(pg_init, pg_n+1)]
    url_list = [ _new_drug.url_drug_revs+ url_c[1] +str(ik) + url_c[2] for ik in range(pg_init, pg_n+1)]
#     print(url_list)
    return url_list

In [5]:
# for a particular drug
# takes url list, scrapes, returns pages of soup
def scraper(url_list):
    soup_list = []
    for url in url_list:
        soup_list.append( [url, load_soup(url)])
    return soup_list 

In [6]:
# for a particular drug
# uses methods from the two parser classes to slice and dice soup into review objects
# should be a "site" method...
def parse_reviews(pages, drug, parser, tag):
#     if parser.abbrev == 'ddc':
#         tag = 'block-wrap comment-wrap'
#     elif(parser.abbrev)== 'wmd':
#         tag = 'userPost'
    for page in pages:
        rev_stew = page[1].find_all('div' ,{'class' : tag})
        for ik, item in enumerate(rev_stew):
            new_review = review(drug, item, parser, ik)
            drug.reviews.append(new_review)

In [7]:
# for a particular drug
# Review object
class review(drug):
    
        def __init__(self, drug, _review_soup, site, ik):
            
            reviewer_info = site.set_reviewerMeta(_review_soup, ik)
            self.drugName = drug.name
            self.site = site.name
            self.condition = site.set_condition(_review_soup)
            self.reviewDate = site.set_reviewDate(_review_soup)
            self.userName = site.set_userName(reviewer_info) #temp.split(',')[0]
            self.ageRange = site.set_ageRange(reviewer_info) #re.search('\s\w+[-]\w+\s', temp).group().strip()
            self.gender = site.set_gender(reviewer_info) #re.split('\s\w+[-]\w+\s', temp)[1].split()[0]
            self.role = site.set_role(_review_soup)
            self.medDuration = site.set_medDuration(reviewer_info) #re.split('on Treatment for ', temp)[1].split('(Patient)')[0].strip()
            self.effectiveness = site.set_effectiveness(_review_soup)
            self.ease_of_use = site.set_ease_of_use(_review_soup)
            self.satisfaction = site.set_satisfaction(_review_soup)
            self.genRating = site.set_genRating(_review_soup)
            self.comment = site.set_comment(_review_soup, ik)
            self.upVotes = site.set_upVotes(_review_soup)


#### Drugs.com

In [8]:
# ************ Drugs.com ************  Parser

# modified from source: 
# https://blog.nycdatascience.com/student-works/web-scraping/anti-epileptic-drug-review-analysis/

class DrugsDotCom:
    
        def __init__(self, name, abbrev):
            self.name = name
            self.abbrev = abbrev
            
        def Drugslist_url_list(self, condition, pg_init, pg_n):
            url_c = ['https://www.drugs.com/condition/', '.html?page_number=']
            url_list = [url_c[0] + condition + url_c[1] +str(ik) for ik in range(pg_init, pg_n+1)]
            return url_list
        
        # function to process soups to extract drug metadata (mix of site specific and standardized) for drugs.com
        def process_drug(self, _drug_summary, _drug_condition, site, _new_drug):
            name_soup = _drug_summary.find('td', {'class':'condition-table__drug-name' })
            _new_drug.name = name_soup.text.strip().split('\n')[0]
            _new_drug.generic = _drug_condition.find('p', {'class': 'condition-table__generic-name'}).text.strip().split('Generic name:\xa0')[1].strip()
            _new_drug.url_drug = 'https://www.drugs.com'+str(name_soup).split('href="')[1].split('" onclick')[0] 
            _new_drug.num_rev = _drug_summary.find('td', {'class':'condition-table__reviews'}).text.replace('reviews', '').strip()
            try:
                _new_drug.num_rev= int(_new_drug.num_rev)
            except:
                _new_drug.num_rev = 0
            _new_drug.num_rev_pages = _new_drug.num_rev//25 + 1
            _new_drug.score = _drug_summary.find('td', {'class': 'condition-table__rating'}).text.strip()
            pop_soup = _drug_summary.find('td', {'class': 'condition-table__popularity'})
            popularity = str(pop_soup.find('div', {'class': 'meter'})).split('width:')[1].split('%')[0]
            _new_drug.url_drug_revs= 'https://www.drugs.com'+ str(_drug_summary.find('td', {'class': 'condition-table__reviews'})).split('href="')[1].split('"')[0]
            return _new_drug
            
        def get_drug_metadata(self, condition, pg_init, pg_n):
            # all drugs for a condition
            #call function to build list of all drugs used to treat depression
            abbrev = self.abbrev
            drugslist_list = self.Drugslist_url_list(condition, pg_init, pg_n)
            if abbrev ==  'ddc':
                # initialize lists for two kinds of soup needed to fill drug metadata fields for ddc
                druglistsummary_soups = []
                druglistprofile_soups = []

                #for each of the drugs in the list of drugs used to treat the condition (in this case depression), get the two kinds of soup
                for url in drugslist_list:
                    soup = load_soup(url)
                    drug_summary = soup.find_all('tr', {'class': 'condition-table__summary'})
                    druglistsummary_soups= druglistsummary_soups+drug_summary
                    drug_profile = soup.find_all('tr', {'class': 'condition-profile'})
                    druglistprofile_soups = druglistprofile_soups + drug_profile

                _drug_list_ds = []
                for ik in range(len(druglistsummary_soups)):
                    new_drug = drug()
                    drug_meta = self.process_drug(druglistsummary_soups[ik], druglistprofile_soups[ik], abbrev, new_drug)
                    _drug_list_ds.append(drug_meta)
#                     print(drug_meta.__dict__)
            return _drug_list_ds
        

        def get_revs_url_list(self, _new_drug):
            cond_soup = load_soup(_new_drug.url_drug_revs)
            mega = str(((cond_soup.find('div', {'id': 'contentWrap'})).find('div', {'class': 'contentBox'})))#.find('div', {'class':'responsive-table-wrap-mobile'})
            options = mega.split('gotoArr[')[2:-1]
            options = [(str(option).split("= '")[1]).split("';\n")[0] for option in options]
            numbers_soup = ((cond_soup.find('div', {'id': 'contentWrap'})).find('div', {'class': 'contentBox'})).find('div', {'class':'data-list-filter'}).find_all('option')[1:]
            rev_cts = [ (str(each).split('(')[1]).split(')<')[0] for each in numbers_soup]

            revs_urls = []
            cond_codes_pgs = []
            total_revs = 0
            for ik in range(len(options)):
                if 'epressi' in options[ik]:
                    cond_codes_pgs.append((options[ik], int(rev_cts[ik])))
            for cond_pg in cond_codes_pgs:
                cond = cond_pg[0]
                pg_n = int(cond_pg[1])//25
                revs_urls = revs_urls+["http://www.drugs.com"+ cond + '/?page='+ str(ik) for ik in range(1, pg_n+1)]
                total_revs +=int(cond_pg[1])
            _new_drug.num_rev_pages = total_revs
            return _new_drug, revs_urls
        
        
        # fetch information about author;
        # bug fix: added a tag to the tags list.  There may be more lurking...
        def set_reviewerMeta (self, _rev_soup, ik):
            tags = ['user-name user-type user-type-2_non_member', 'user-name user-type user-type-1_standard_member','user-name user-type user-type-0_select_member']
            if _rev_soup.find('p', {'class': tags[0]}):
                return _rev_soup.find('p', {'class': tags[0]})
            elif _rev_soup.find('p', {'class': tags[1]}):
                return _rev_soup.find('p', {'class': tags[1]})
            elif _rev_soup.find('p', {'class': tags[2]}):
                return _rev_soup.find('p', {'class': tags[2]})
            else:
                return None
            
        def set_userName (self, _reviewerMeta):
                try:
                    return _reviewerMeta.contents[0].strip()
                except:
                    return 'Anonymous'

                
        #need to fix this        
        def set_ageRange (self, _reviewerMeta):
                try:
                    return re.search('\s\w+[-]\w+\s', _reviewerMeta).group().strip()
                except:
                    return None
                    
                    
        #gender not specified on drugs.com
        def set_gender (self, _reviewerMeta):
                return None
            
        #role not specified on drugs.com
        def set_role(self, _rev_soup):
            return None
            
        def set_medDuration (self, _reviewerMeta):
                try: 
                    dates =_reviewerMeta.find_all('span')#, {'class':'small light'})
                    if len(dates)>1:
                        return str(dates[0]).split('<span class="tiny light">(taken for')[1].split(')</span>')[0].strip()
                except:
                    return None

                
        def set_reviewDate (self, _reviewerMeta):
                try:
                    dates =_reviewerMeta.find_all('span', {'class':"tiny light comment-date"})
                    if len(dates)>1:
                        return str(dates[1]).split('<span class="tiny light comment-date">')[1].split('</span>')[0]
                    else:
                        return str(dates[0]).split('<span class="tiny light comment-date">')[1].split('</span>')[0]
                except:
                    return None
                
                
        def set_condition (self, _rev_soup):
                try:
                    return _rev_soup.find('div', {'class':'user-comment'}).b.get_text()
                except:
                    return None
                
                
        #from WebMD.com
        def set_effectiveness (self, _rev_soup):
                return None
            
            
        #from WebMD.com
        def set_ease_of_use (self, _rev_soup):
                return None
            
            
        #from WebMD.com
        def set_satisfaction (self, _rev_soup):
                return None

            
        def set_genRating (self, _rev_soup):        
                try:
                    return int(_rev_soup.find('div',{'class': 'rating-score'}).get_text())
                except:
                    return None


        def set_comment (self, _rev_soup, ik):
                try:
                    return _rev_soup.find('div', {'class':'user-comment'}).span.get_text()
                except:
                    return None


        def set_upVotes (self, _rev_soup):
                try: 
                    return int(_rev_soup.find_all('p', {'class':"tiny light"})[0].b.get_text().split(' users')[0])
                except:
                    return None


In [13]:
# ************ webMD.com ************ Parser

class WebMD:
    
        def __init__(self, name, abbrev):
            self.name = name
            self.abbrev = abbrev
            
        #only one     
        def Drugslist_url_list(self, condition, pg_init, pg_n):
            url_list = ['https://www.webmd.com/drugs/2/condition-952/major%20depressive%20disorder']#, 
                        #'https://www.webmd.com/drugs/2/condition-1022/depression', 
                        #'https://www.webmd.com/drugs/2/condition-13493/depression%20treatment%20adjunct']
            return url_list
        
        # function to process soups to extract drug metadata
        def process_drug(self, _drug_summary, site, _new_drug):
            drug_data = _drug_summary.find_all('td')
            _new_drug.name = drug_data[0].get_text()
            _new_drug.url_drug = 'http://www.webmd.com'+str(drug_data[0]).split('a href="')[1].split('">')[0]
            name_soup = load_soup(_new_drug.url_drug)
            _new_drug.generic = str(name_soup.find('section', {'class':'generic-name'}).find('p')).split('</span>')[1].split('</p>')[0]
            _new_drug.num_rev = int((drug_data[3].get_text().split(' Reviews')[0]))
            _new_drug.num_rev_pages = _new_drug.num_rev//5 + 1
            _new_drug.url_drug_revs=  'http://www.webmd.com'+str(drug_data[3]).split('a href="')[1].split('">')[0]#.split('href="')[1].split('"')[0]
#             print(_new_drug.url_drug_revs[site])
            return _new_drug
            
        def get_drug_metadata(self, condition, pg_init, pg_n):
            # all drugs for a condition
            #call function to build list of all drugs used to treat depression
            abbrev = self.abbrev
            drugslist_list = self.Drugslist_url_list(condition, pg_init, pg_n)
            # initialize lists for two kinds of soup needed to fill drug metadata fields for ddc
            druglistsummary_soups = []


            #for each of the drugs in the list of drugs used to treat the condition (in this case depression), get the two kinds of soup
            for url in drugslist_list:
                soup = load_soup(url)
                drug_summary = ((soup.find('table', {'class':'drugs-treatments-table'})).find('tbody')).find_all('tr')
                druglistsummary_soups= druglistsummary_soups+drug_summary

            _drug_list_ds = []
            for ik in range(len(druglistsummary_soups)):
                new_drug = drug()
                drug_meta = self.process_drug(druglistsummary_soups[ik], abbrev, new_drug)
                _drug_list_ds.append(drug_meta)
#                 print(drug_meta.__dict__)
            return _drug_list_ds

        def get_revs_url_list(self, _new_drug):
            cond_soup = load_soup(_new_drug.url_drug_revs)
            print(_new_drug.url_drug_revs)
            cond_codes_pgs= []
            options = (cond_soup.find('select', {'id':'conditionFilter'})).find_all('option')
            for option in options:
                if 'epressi' in option.text:
                    cond_codes_pgs.append((str(option).split('value="')[1].split('"')[0], option.text.split('(')[1].split(' reviews')[0]))
            revs_urls = []
            total_revs = 0
            for cond_pg in cond_codes_pgs:
                cond = cond_pg[0]
                pg_n = int(cond_pg[1])//5
#     url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) + url_c[2] for ik in range(pg_init, pg_n+1)]
                revs_urls = revs_urls+ [ _new_drug.url_drug_revs+ '&pageIndex=' +str(ik) + '&sortby=3'+'&conditionFilter='+str(cond) for ik in range(0, pg_n+1)]
#                 url_list = [ _new_drug.url_drug_revs[site]+ url_c[1] +str(ik) + url_c[2]+url_c[3] for ik in range(pg_init, pg_n+1)]
#                 revs_urls = revs_urls+revs_url_list(new_drug, start_num, pg_n, WMD_parser.abbrev, cond)
                total_revs +=int(cond_pg[1])
            _new_drug.num_rev = total_revs
            return _new_drug, revs_urls
        
            
        def set_reviewerMeta (self, _rev_soup, ik):
            try:
                return _rev_soup.find('p', {'class':'reviewerInfo'}).text.strip('Reviewer: ')
            except:
                return None
            
        #below takes reviewer soup
        def set_userName (self, _reviewerMeta):
            try:
                splits = _reviewerMeta.split(',')
                if len(splits)>1:
                    return splits[0]
                else:
                    return 'Anonymous'
            except:
                return 'Anonymous'

        def set_ageRange (self, _reviewerMeta):
            try:
                return re.search('\s\w+[-]\w+\s', _reviewerMeta).group().strip()
            except:
                return None

        def set_gender (self, _reviewerMeta):
            try: 
                return re.split('\s\w+[-]\w+\s', _reviewerMeta)[1].split()[0]
            except: 
                return None
        
        def set_role(self, _rev_soup):
            try:
                return _rev_soup.find('p', {'class':'reviewerInfo'}).text.strip('Reviewer: ').split(' ')[-1].replace('(','').replace(')','')
            except:
                return None
            
        def set_medDuration (self, _reviewerMeta):
            try:
                return re.split('on Treatment for ', _reviewerMeta)[1].split('(Patient)')[0].strip()
            except:
                return None
            
        #below takes full soup
        #untested for webMD
        def set_reviewDate  (self, _rev_soup):
            try:
                return _rev_soup.find('div', {'class': 'date'}).text.split(' ',1)[0]

            except:
                return None
                
        def set_condition (self, _rev_soup):
            try:
                condition = _rev_soup.find('div', {'class': 'conditionInfo'}).text
                temp = condition.split('Condition: ')[1]
                return temp
            except:
                return None
                
        def set_effectiveness (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings firstEl clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

        def set_ease_of_use (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

        def set_satisfaction (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings lastEl clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None
                
        #from drugs.com        
        def set_genRating  (self, _rev_soup):
                    return None

        def set_comment (self, _rev_soup, ik):
                try: 
                    temp = _rev_soup.find('p', {'id':'comFull'+str(ik+1)}).text
                    temp = re.split('Hide Full', temp)[0]
                    return temp.lstrip('Comment:')
                except:
                    return None

        def set_upVotes (self, _rev_soup):
                try:
                    temp = _rev_soup.find('p', {'class' : "helpful"}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

In [14]:
def build_depression_drugs(site, pickleopt, picklename):
    # Set condition
    condition = 'depression'
    def doItAll(parser, start_num, tag):
        start = time.time()
        all_drugs_list = parser.get_drug_metadata('depression', 1, 1)
        print('got list:', time.time()-start)
        drug_list = []
        generics_list = [new_drug.generic.strip(' systemic') for new_drug in all_drugs_list]
        print('number of generics:', len(generics_list))

        for new_drug in all_drugs_list:
            if (new_drug.name in generics_list) or (new_drug.num_rev>=200):
                print(new_drug.name)
#                 new_drug_Soup = scraper([new_drug.url_drug_revs[parser.abbrev]])
                new_drug, revs_url_list = parser.get_revs_url_list(new_drug)
#                 print(revs_url_list)
                new_drug_Soup = scraper(revs_url_list)
                print('scraped sites:', time.time()- start)
                parse_reviews(new_drug_Soup, new_drug, parser, tag)
                print('parsed reviews:', time.time()- start)
                drug_list.append(new_drug)
                print('number of drugs on short list so far:', len(drug_list))
#             print(len(new_drug.reviews[parser.abbrev]))
            print('got first drug:', time.time())
        return drug_list, all_drugs_list, generics_list

    if site == 'ddc':
        # Initialize site objects
        DDC_parser = DrugsDotCom('Drugs_dot_com', 'ddc')
        tag = 'block-wrap comment-wrap'
        filled_drug_list, all_drugs_list, generics_list = doItAll(DDC_parser,0, tag)
        
    elif site == 'wmd':
        WMD_parser = WebMD('WebMD', 'wmd')
        tag = 'userPost'
        filled_drug_list, all_drugs_list, generics_list = doItAll(WMD_parser,1, tag)
        
    if pickleopt =='y':
        pickle.dump( filled_drug_list, open( picklename+'.p', "wb" ) )
        
    return filled_drug_list, all_drugs_list, generics_list
        

In [15]:
filled_drugs_ddc, all_drugs_ddc, generics_list_ddc  = build_depression_drugs('ddc', 'n', 'drug_list_ddc')

got list: 1.6132330894470215
number of generics: 25
Cymbalta
scraped sites: 14.665996789932251
parsed reviews: 16.321555137634277
number of drugs on short list so far: 1
got first drug: 1511631686.941966
Zoloft
scraped sites: 30.468865871429443
parsed reviews: 31.79347801208496
number of drugs on short list so far: 2
got first drug: 1511631702.41396
Lexapro
scraped sites: 44.72217106819153
parsed reviews: 46.035006046295166
number of drugs on short list so far: 3
got first drug: 1511631716.6553621
Prozac
scraped sites: 55.749008893966675
parsed reviews: 56.58011817932129
number of drugs on short list so far: 4
got first drug: 1511631727.200969
Celexa
scraped sites: 66.5448169708252
parsed reviews: 67.44801807403564
number of drugs on short list so far: 5
got first drug: 1511631738.068473
bupropion
scraped sites: 91.179025888443
parsed reviews: 93.82261991500854
number of drugs on short list so far: 6
got first drug: 1511631764.442925
citalopram
scraped sites: 114.50709104537964
parsed 

In [16]:
pickle.dump( filled_drugs_ddc, open( "drug_list_ddc2.p", "wb" ) )

In [53]:
drug_list_ddc2 = drug_list_ddc
del drug_list_ddc, new_drug

In [16]:
drug_list_ddc = pickle.load( open("drug_list_ddc.p", "rb"))

#### WebMD

In [ ]:
drug_list_wmd, all_drugs_wmd, generics_list_md = build_depression_drugs('wmd', 'n', 'drug_list_wmd')

In [80]:
pickle.dump(drug_list_wmd, open("drug_list_wmd.p", 'wb' ))

In [ ]:
ddc_names = [(drug.name).lower() for drug in drug_list_ddc]
wmd_names = [(drug.name).lower() for drug in drug_list_wmd]

crossover = []
for drug_ddc in ddc_names:
    if drug_ddc in wmd_names:
        crossover.append(drug_ddc)
        
print(crossover)

In [102]:
from collections import Counter
rev_list = []
age_list = []

reviews = wmd_dicts['Abilify'].reviews['wmd']

for review in reviews:
    rev_list.append(str(review.medDuration))
    age_list.append(str(review.ageRange))

print(type(rev_list[0][0]))
print(Counter(rev_list))
print(Counter(age_list))

<class 'str'>
Counter({'[1, 6]': 1, '[24, 60]': 1, '1 to less than 2 years': 1, '[0, 1]': 1, '[60, 120]': 1})
Counter({'[65, 74]': 2, '[55, 64]': 2, 'None': 1})


In [103]:
d2 = enchant.DictWithPWL("en_US","new_words.txt")

In [104]:
d2.check('wikkipedia')

True

In [108]:
# drug_list = ['Cymbalta', 'Zoloft', 'Lexapro', 'Prozac', 'Celexa', 'bupropion', 'citalopram', 'Wellbutrin XL', 'sertraline', 'Abilify', 'fluoxetine', 'Xanax', 'Wellbutrin', 'Effexor XR', 'venlafaxine', 'Pristiq', 'Effexor', 'escitalopram', 'Paxil', 'trazodone', 'Wellbutrin SR', 'mirtazapine', 'Remeron', 'Viibryd', 'duloxetine', 'paroxetine', 'Seroquel XR', 'amitriptyline', 'alprazolam', 'Deplin', 'lamotrigine', 'Zyprexa', 'nortriptyline', 'quetiapine', 'aripiprazole', 'Desyrel', 'Fetzima', 'tramadol', 'Alprazolam Intensol', 'Budeprion XL', 'Niravam', 'Oleptro', 'Trintellix', 'Aplenzin', 'Budeprion SR', 'Desyrel Dividose', 'doxepin', 'Forfivo XL', 'olanzapine', 'Paxil CR', 'Prozac Weekly', 'risperidone', 'Serzone', 'imipramine', 'lithium', 'Rapiflux', 'Asendin', 'desipramine', 'desvenlafaxine', 'Emsam', 'Irenka', 'l-methylfolate', 'Methylin ER', 'methylphenidate', 'Nardil', 'nefazodone', 'niacin', 'Pamelor', 'Parnate', 'Pexeva', 'Remeron SolTab', 'Sinequan', 'Symbyax', 'vortioxetine', 'amitriptyline', 'chlordiazepoxide', 'amoxapine', 'Anafranil', 'Aventyl Hydrochloride', 'clomipramine', 'fluoxetine / olanzapine', 'fluvoxamine', 'Limbitrol', 'Limbitrol DS', 'modafinil', 'Norpramin', 'paliperidone', 'Rexulti', 'selegiline', 'Tofranil', 'vilazodone', 'Zyprexa Zydis', 'amitriptyline','perphenazine', 'armodafinil', 'atomoxetine', 'brexpiprazole', 'Duo-Vil', 'Etrafon', 'Etrafon Forte', 'isocarboxazid', 'Khedezla', 'L-Methylfolate Formula', 'L-Methylfolate Forte', 'levomilnacipran', 'lisdexamfetamine', 'Ludiomil', 'maprotiline', 'Marplan', 'phenelzine', 'protriptyline', 'Surmontil', 'Tofranil-PM', 'tranylcypromine', 'Triavil', 'trimipramine', 'Vivactil', 'XaQuil XR']
drug_list_all = ['hctz','ssri','snri','xr','suboxone','respirdal', 'meth','geodon','benztropine', 'valium','lyrica','melatonin','lamictal','depakote','cogentin','neurontin','nexium','hydralazine','topamax', 'ambien','provigil', 'mirapex', 'saphris','miralax','zolpidem', 'Percocet', 'adderall','risperdal','buspirone', 'lorazepam', 'ativan', 'lunesta','vistaril', 'Strattera','Clonazepam','Savella' , 'Pamelor', 'Paxil CR', 'Endep tablet', 'Irenka', 'Trintellix', 'Serzone', 'Remeron', 'clomipramine HCL', 'Viibryd', 'fluvoxamine', 'Pexeva', 'Desyrel', 'nefazodone', 'Norfranil tablet', 'Limbitrol', 'Elavil Solution', 'fluvoxamine MALEATE ER', 'Oleptro', 'doxepin HCL', 'nortriptyline', 'L-Methylfolate Forte', 'imipramine', 'Alprazolam Intensol', 'trazodone', 'vortioxetine', 'Seroquel','Ludiomil', 'escitalopram', 'Emsam', 'Etnofril tablet', 'Limbitrol DS', 'Forfivo XL', 'L-Methylfolate Formula', 'Asendin tablet', 'doxepin', 'Khedezla', 'trimipramine MALEATE', 'Budeprion SR', 'Fetzima', 'Rexulti', 'risperidone', 'protriptyline', 'E-Vill 50 tablet', 'Symbyax', 'Nardil', 'Sinequan Concentrate', 'desipramine', 'Deconil tablet', 'Triavil', 'citalopram', 'Norpramin', 'desipramine HCL', 'E-Vill 100 tablet', 'paliperidone', 'duloxetine', 'Sk-Pramine tablet', 'Budeprion XL', 'Amitid tablet', 'E-Vill 25 tablet', 'bupropion', 'venlafaxine', 'Tofranil', 'Sinequan', 'Parnate', 'Vivactil', 'isocarboxazid', 'amitriptyline HCL', 'Adapin capsule', 'atomoxetine', 'doxepin HCL capsule', 'alprazolam', 'desvenlafaxine', 'Anafranil', 'lamotrigine', 'tramadol', 'Janimine tablet', 'imipramine HCl', 'lisdexamfetamine', 'Rapiflux', 'Stabanil tablet', 'Paxil', 'aripiprazole', 'Aplenzin', 'amitriptyline / chlordiazepoxide', 'Remeron SolTab', 'sertraline', 'Aventyl Hydrochloride', 'Aventyl capsule', 'Prozac Weekly', 'l-methylfolate', 'Elavil tablet', 'Lexapro', 'Duo-Vil', 'Niravam', 'phenelzine SULFATE', 'armodafinil', 'trimipramine', 'amoxapine', 'Surmontil', 'amitriptyline Solution', 'amitriptyline', 'Effexor XR', 'Methylin ER', 'Vanatrip tablet', 'quetiapine', 'selegiline', 'amitriptyline / perphenazine', 'phenelzine', 'Wellbutrin XL', 'E-Vill 75 tablet', 'Tofranil-PM', 'Emitrip tablet', 'Enovil Solution', 'levomilnacipran', 'XaQuil XR', 'vilazodone', 'Luvox CR', 'Effexor', 'brexpiprazole', 'Zoloft', 'imipramine pamoate', 'protriptyline HCL', 'Etrafon', 'Luvox tablet', 'niacin', 'thyroid desiccated', 'Xanax', 'maprotiline', 'Pristiq', 'E-Vill 10 tablet', 'Wellbutrin', 'doxepin tablet', 'Celexa', 'Sinequan capsule', 'Abilify', 'Deplin', 'maprotiline HCL', 'Marplan', 'Imavate tablet', 'tranylcypromine', 'lithium', 'Ludiomil tablet', 'paroxetine', 'Wellbutrin SR', 'olanzapine', 'Prozac', 'modafinil', 'Kenvil tablet', 'Zyprexa Zydis', 'Zyprexa', 'Cymbalta', 'Re-Live tablet', 'fluvoxamine MALEATE', 'Asendin', 'clomipramine', 'Q.E.L tablet', 'methylphenidate', 'fluoxetine', 'Desyrel Dividose', 'Etrafon Forte', 'Seroquel XR', 'fluoxetine / olanzapine', 'mirtazapine', 'nortriptyline HCL']
drug_list = [word.lower() for word in drug_list_all]
# if we need more drug names... https://druginfo.nlm.nih.gov/drugportal/drug/names

adds = ['weepiness','dr', 'meds','mdd','aspergers','crittenden','walmart','wikkipedia', 'dysthymia','ocd','apnea','ptsd', 'occ','pancreatitis','electroconvulsive','neuropathy','mgs','antipsychotic','thyrotoxicosis','oculogyric','ADHD','pychotic','tricyclic','mitral', 'dyskinesia', 'hypomanic','hypomania', 'dystonic', 'tardive', 'Pristq', 'fibromyalgia', 'akathesia' ]
adds = [word.lower() for word in adds]

new_words = drug_list+adds

In [111]:
error_list = []
error_ref = []

chkr = SpellChecker("en_US", filters=[EmailFilter,URLFilter])

def spellchecker(reviews):
    for ik, review in enumerate(reviews):
    #         print ('-------------')
    #         print (review['comment'])
    #         print(ftfy.fix_text(review['comment'])#['review'])
    #         words = [word.lower() for word in words]
            chkr.set_text(review.comment)
            for err in chkr:
                error = (err.word).lower()
                options = [change.lower() for change in d2.suggest(error)]
                if error in options:
                    continue
                else:
                    catch = 'no'
                    word_to_beat = options[0]
                    score_to_beat = 10
                    for word in options:
                        dist = nltk.edit_distance(error, word)
                        if (word in new_words):# and (nltk.edit_distance(error, word)<score_to_beat):
                            if (nltk.edit_distance(error, word)<=score_to_beat):
                                word_to_beat = word
                                score_to_beat = nltk.edit_distance(error, word)
                                catch = 'yes'
                        elif dist<score_to_beat:
                            word_to_beat = word
                            score_to_beat = nltk.edit_distance(error, word)
                    print('final option:', error, word_to_beat, score_to_beat)

In [112]:
spellchecker(reviews)

final option: dont dot 1
final option: hurrah hurray 1


Two test cases below:
    1.) Abilify with WebMD
    2.) Methylphenidate with Drugs.com
    
the code uses url_stem and range of pages to generate a url list, then goes and scrapes those pages,
and parses them into reviews

### WebMD Abilify test case

In [ ]:
# retired
# builds url lists specific to the site
def WebMD_url_list(drug_stem, pg_init, pg_n):
    url_c = ['https://www.webmd.com/drugs/drugreview-', '&pageIndex=', '&sortby=3&conditionFilter=-500' ]
    url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) + url_c[2] for ik in range(pg_init, pg_n+1)]
    return url_list
    

In [ ]:
# Get Abilify Soup
abilify_stem = '64439-Abilify-oral.aspx?drugid=64439&drugname=Abilify-oral'
abilify = drug('abilify', abilify_stem)
abilify_Soup = scraper(WebMD_url_list(abilify.url_stem, 0, 140))

In [ ]:
# Parse Abilify Soup
abilify = drug('abilify', abilify_stem)
webMD_parser = WebMD('webMD')
webMD_tag = 'userPost'
parse_reviews(abilify_Soup, webMD_tag, abilify, webMD_parser)

In [ ]:
# prints review objects for inspection

for reviewx in abilify.reviews[0:10]:
    for key in reviewx.__dict__:
        print(key, ':', reviewx.__dict__[key])
    print('----------')
    print('')

In [ ]:
#trouble shoot patient metadata
abilify_Soup[0][1].find('p', {'class':'reviewerInfo'}).text
rev_stew[1].find('p', {'class':'reviewerInfo'}).text

In [ ]:
for page in abilify_Soup:
        rev_stew = page[1].find_all('div' ,{'class' : webMD_tag})
        for ik, item in enumerate(rev_stew):
            print(item)
            break

### Drugs.com Meth test case

In [ ]:
# Drugs.com test

meth_stem = 'methylphenidate'
meth = drug('methylphenidate', meth_stem)
meth_Soup = scraper(Drugs_url_list(meth.url_stem, 0, 2))

In [ ]:
meth = drug('methylphenidate', meth_stem)
drugs_tag = 'block-wrap comment-wrap'
drugs_parser = DrugsDotCom('drugsDotCom')
parse_reviews(meth_Soup, drugs_tag, meth, drugs_parser)

In [ ]:
# Pickle Soup (at the very least--work on pickling objects (?) later)

In [ ]:
dict_list = []
for reviewx in abilify.reviews:
    dict_list.append(reviewx.__dict__)

In [ ]:
pickle.dump( dict_list, open( "abilify.p", "wb" ) )

### Side project: scraping drug list

In [ ]:
# url = 'https://www.drugs.com/condition/depression.html'


In [ ]:
def Drugslist_url_list(condition, pg_init, pg_n):
    url_c = ['https://www.drugs.com/condition/', '.html?page_number=']
    url_list = [url_c[0] + condition + url_c[1] +str(ik) for ik in range(pg_init, pg_n+1)]
    return url_list

In [ ]:
drugslist_list = Drugslist_url_list('depression', 1, 5)

In [ ]:
druglistsummary_soups = []
druglistprofile_soups = []

In [ ]:
for url in drugslist_list:
    soup = load_soup(url)
    drug_summary = soup.find_all('tr', {'class': 'condition-table__summary'})
    druglistsummary_soups= druglistsummary_soups+drug_summary
    drug_profile = soup.find_all('tr', {'class': 'condition-profile'})
    druglistprofile_soups = druglistprofile_soups + drug_profile

In [ ]:
drug_summary[-5].find('td', {'class':'condition-table__drug-name' }).text.strip().split('\n')[0]

In [ ]:
def process_drug(_drug_summary, _drug_condition):
    name_soup = _drug_summary.find('td', {'class':'condition-table__drug-name' })
    name = name_soup.text.strip().split('\n')[0]
    url_drug = 'https://www.drugs.com'+str(name_soup).split('href="')[1].split('" onclick')[0]
    num_rev = _drug_summary.find('td', {'class':'condition-table__reviews'}).text.replace('reviews', '').strip()
    score = _drug_summary.find('td', {'class': 'condition-table__rating'}).text.strip()
    pop_soup = _drug_summary.find('td', {'class': 'condition-table__popularity'})
    popularity = str(pop_soup.find('div', {'class': 'meter'})).split('width:')[1].split('%')[0]
    generic = _drug_condition.find('p', {'class': 'condition-table__generic-name'}).text.strip().split('Generic name:\xa0')[1].strip()
    drug_comments_url = drug_soup.find('div', {'id': 'content-box-nav-tabs'})
    lis = drug_comments_url.find_all('li')
    for li in lis:
        if li.text == 'User Reviews':
            url_drug_rev= 'https://www.drugs.com'+ r(li).split('href="')[1].split('" onclick')[0]
    drug_d = {'name':name, 'generic': generic, 'url_drug': url_drug, 'url_drug_rev': url_drug_rev, 'num_rev':num_rev, 'score':score, 'popularity': popularity}
    return drug_d

In [ ]:
drug_list_ds = []
for ik in range(len(druglistmeta_soups)):
    drug_meta = process_drug(druglistsummary_soups[ik], druglistprofile_soups[ik])
#     drug_soup = load_soup(drug_meta['url_drug'])
    print(drug_meta['name'], '(', drug_meta['generic'], ')')
    drug_list_ds.append(drug_meta)


In [ ]:
# this list reflects the list of all user initiated categories on drugs.com for depression
# the next question is: which should be merged because the generic and brand are discussed seperately
for ik in range(len(drug_list_ds)):
    print(drug_list_ds[ik]['name'],': ', drug_list_ds[ik]['num_rev'])

#### Merging (or not) generics and brand

In [ ]:
# now unnecessary
def Drugs_url_list(drug_stem, pg_init, pg_n):
    url_c = ['https://www.drugs.com/comments/', '/?page=']
    url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) for ik in range(pg_init, pg_n+1)]
    return url_list

In [ ]:
# pulling reviews url from drug info page
drug_soup = load_soup(drug_list_ds[0]['url_drug'])
drug_comments_url = drug_soup.find('div', {'id': 'content-box-nav-tabs'})
lis = drug_comments_url.find_all('li')
for li in lis:
    if li.text == 'User Reviews':
        rev_sffx = str(li).split('href="')[1].split('" onclick')[0]


In [ ]:
#generic = 
(drug_soup.find('p', {'class': 'drug-subtitle'})).text.split("Generic Name: ")[1].split(' ')[0]

In [ ]:
#brand_names = 
(drug_soup.find('p', {'class': 'drug-subtitle'})).text.split("Generic Name: ")[1].split(')')[1].split('Brand Names: ')[1].split(', ')

In [ ]:
#webmd process each drug meta data from drug list
def process_drug(_drug_summary, _drug_condition, site, _new_drug):
    drug_data = _drug_summary.find_all('td')
    _new_drug.name = drug_data[0].get_text()
    _new_drug.url_drug[site] = 'http://www.webmd.com/'+str(drug_data[0]).split('a href="')[1].split('">')[0]
    name_soup = load_soup(drug_url)
    _new_drug.generic = str(name_soup.find('section', {'class':'generic-name'}).find('p')).split('</span>')[1].split('</p>')[0]
    _new_drug.num_rev[site] = (drug_data[3].get_text().split(' Reviews')[0])
    _new_drug.url_drug_revs[site]=  'http://www.webmd.com/'+str(drug_data[3]).split('a href="')[1].split('">')[0].split('href="')[1].split('"')[0]
    return _new_drug

In [ ]:
#webmd hasn't been written 
# fyi: https://www.webmd.com/depression/guide/depression-medications-antidepressants

depression_soup = load_soup('https://www.webmd.com/drugs/2/condition-1022/depression')
drug_list = ((depression_soup.find('table', {'class':'drugs-treatments-table'})).find('tbody')).find_all('tr')

for drug in drug_list:
    drug_data = drug.find_all('td')
    name = drug_data[0].get_text()
    drug_url = 'http://www.webmd.com/'+str(drug_data[0]).split('a href="')[1].split('">')[0]
    name_soup = load_soup(drug_url)
    num_revs = (drug_data[3].get_text().split(' Reviews')[0])
    drug_revs_url = str(drug_data[3]).split('a href="')[1].split('">')[0]
    break